In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np

In [ ]:
df_fesx = pd.read_csv("./data/fesx_M15_2020-06-20_F.US.DSXM20.scid_BarData.txt", sep=", ", engine = "python")
df_fesx["Datetime"] = pd.to_datetime(df_fesx.Date + "-" + df_fesx.Time,format="%Y/%m/%d-%H:%M:%S")
df_fesx = df_fesx[['Datetime','Open', 'High', 'Low', 'Last', 'Volume']].rename(columns={"Last":"Close"})#.set_index("Datetime")
df_fesx = df_fesx.rename(columns={"Last":"Close"})
df_fesx.head()

In [ ]:
# Detect when the price goes up 5 consecutive times by at least 2 
df_fesx['Diff'] = df_fesx.High.diff()

rolling_n = 3
min_diff = 0

df_fesx['DiffMin'] = df_fesx.Close.diff().rolling(rolling_n).min()
df_fesx['DiffMax'] = df_fesx.Close.diff().rolling(rolling_n).max()

df_fesx['Buy'] = False
df_fesx['Sell'] = False

mask_buy = (df_fesx.DiffMin> min_diff)
mask_sell = (df_fesx.DiffMax < -min_diff)

df_fesx.loc[mask_buy, 'Buy'] = True
df_fesx.loc[mask_sell, 'Sell'] = True

fig = go.Figure(data= [
    go.Scatter(x=np.arange(df_fesx.shape[0]), y=df_fesx.High,name = 'fesx',mode="lines+markers"),
    go.Scatter(x=np.arange(df_fesx.shape[0]), y=df_fesx.High.rolling(30).mean(), name=f'RM({30})'),
    go.Scatter(x=df_fesx[mask_buy].index, y=df_fesx[mask_buy].High, mode='markers', marker_color='red', name = 'Buy'),
    go.Scatter(x=df_fesx[mask_sell].index, y=df_fesx[mask_sell].High, mode='markers', marker_color='green', name= 'Sell'),

    go.Candlestick(x=np.arange(df_fesx.shape[0]),
                open=df_fesx['Open'],
                high=df_fesx['High'],
                low=df_fesx['Low'],
                close=df_fesx['Close'])
    ]            
    )

fig.update_layout(
    xaxis_rangeslider_visible= False
)


In [ ]:
def consec_n_up(arr:pd.Series,n, min_diff):
    arr = pd.Series(arr)
    return arr.iloc[-(n+1):].diff().min()>min_diff

def consec_n_down(arr:pd.Series,n, min_diff):
    arr = pd.Series(arr)
    return arr.iloc[-(n+1):].diff().max()<-min_diff

def fesx(arr : pd.Series):
    return arr

In [ ]:
ns =np.arange(1,11)
min_diffs = np.linspace(0,5,10)
profit = np.zeros((ns.shape[0], min_diffs.shape[0]))

In [ ]:
for n_idx, n in enumerate(ns):
    for min_diff_idx, min_diff in enumerate(min_diffs):
        class Teste1(Strategy):
                def init(self):
                    pass
                #    high = self.data.High
                #    self.array = self.I(fesx, high,scatter=False)
                def next(self):
                    if consec_n_down(self.data.High, n, min_diff):
                        self.sell()
                    if consec_n_up(self.data.High, n, min_diff):
                        self.buy()


        bt = Backtest(df_fesx, Teste1,
                    cash=10000, commission=.002,
                    exclusive_orders=True)

        output = bt.run()
        profit[n_idx][min_diff_idx] = output['Profit Factor']
        raise